In [ ]:
import sys
print(sys.path)
sys.path.append("C:/Users/jz421/Desktop/GlobalLocal/IEEG_Pipelines/") #need to do this cuz otherwise ieeg isn't added to path...

from ieeg.navigate import channel_outlier_marker, trial_ieeg, crop_empty_data, \
    outliers_to_nan
from ieeg.io import raw_from_layout, get_data
from ieeg.timefreq.utils import crop_pad
from ieeg.timefreq import gamma
from ieeg.calc.scaling import rescale
import mne
import os
import numpy as np
import pandas as pd
from ieeg.calc.reshape import make_data_same
from ieeg.calc.stats import time_perm_cluster

import utils
import matplotlib.pyplot as plt

from pandas import read_csv
import scipy.stats as stats
import joblib

from scipy.ndimage import label

import json
import pickle

make model rdms here (just do congruency and switch type for testing for now 5/4)

In [ ]:
# Define the conditions
conditions = ["i25s25", "i25s75", "i75s25", "i75s75", "i25r25", "i25r75", "i75r25", "i75r75",
              "c25s25", "c25s75", "c75s25", "c75s75", "c25r25", "c25r75", "c75r25", "c75r75"]

# Extract specific features from each condition
congruency = [cond[0] for cond in conditions]
switchTypes = [cond[-3] for cond in conditions]
congruencyProportion = [int(cond[1:3]) for cond in conditions]
switchProportion = [int(cond[-2:]) for cond in conditions]

# Number of conditions
n = len(conditions)

# initialize dict to store model RDMs
model_rdms = {'congruency': np.ones((n, n)), 'switchType': np.ones((n, n)),
              'congruencyProportion': np.ones((n, n)), 'switchProportion': np.ones((n, n)), 
              'congruency:congruencyProportion': np.ones((n, n)), 'switchType:switchProportion': np.ones((n,n)),
              'congruency:switchProportion': np.ones((n, n)), 'switchType:congruencyProportion': np.ones((n,n))}

# Populate RDMs based on feature comparisons
for i in range(n):
    for j in range(n):
        if congruency[i] == congruency[j]:
            model_rdms['congruency'][i, j] = 0
        if switchTypes[i] == switchTypes[j]:
            model_rdms['switchType'][i, j] = 0
        if congruencyProportion[i] == congruencyProportion[j]:
            model_rdms['congruencyProportion'][i, j] = 0
        if switchProportion[i] == switchProportion[j]:
            model_rdms['switchProportion'][i, j] = 0
        if congruency[i] == congruency[j] and congruencyProportion[i] == congruencyProportion[j]:
            model_rdms['congruency:congruencyProportion'][i,j] = 0
        if switchTypes[i] == switchTypes[j] and switchProportion[i] == switchProportion[j]:
            model_rdms['switchType:switchProportion'][i,j] = 0
        if congruency[i] == congruency[j] and switchProportion[i] == switchProportion[j]:
            model_rdms['congruency:switchProportion'][i,j] = 0
        if switchTypes[i] == switchTypes[j] and congruencyProportion[i] == congruencyProportion[j]:
            model_rdms['switchType:congruencyProportion'][i,j] = 0

#### load in epochs data  
(get only acc trials, avg across epochs to make evoked using only acc trials, and then delete epochs from memory) 5/5

In [ ]:
subjects = ['D0057','D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077', 'D0090', 'D0094', 'D0100', 'D0102', 'D0103']

make or load subjects electrodes to rois dict (mapping from electrode names to roi labels)

In [ ]:
# load in subjects electrodes to rois dict. If it doesn't already exist, make it and then load it.
filename = 'subjects_electrodestoROIs_dict.json'
subjects_electrodestoROIs_dict = utils.make_or_load_subjects_electrodes_to_rois_dict(filename, subjects)

load accuracy arrays so we can filter by only accurate trials  

In [ ]:
# this makes numpy arrays for each subject that are 0 or 1 for each trial based on accuracy
from makeRawBehavioralData import main
main()

# Directory where your .npy files are saved
npy_directory = r'C:\Users\jz421\Box\CoganLab\D_Data\GlobalLocal\accArrays'  # Replace with your directory path if you're not Jim

acc_array = utils.load_acc_arrays(npy_directory, skip_subjects=['D107'])

load in behavioral data and convert block types to congruency and switch proportions

In [ ]:
combined_data = pd.read_csv(r'C:\Users\jz421\Box\CoganLab\D_Data\GlobalLocal\combinedData.csv')

# Apply the function to each row and create new columns
combined_data[['congruencyProportion', 'switchProportion']] = combined_data.apply(utils.map_block_type, axis=1)

load epochs and evoked for all 16 conditions

test these functions to just load in HG_ev1_rescaled cuz memory issues 5/6. Replace the ones in utils.py if these work.

In [ ]:
def load_mne_objects(sub, output_name, task, just_HG_ev1_rescaled=False, LAB_root=None):
    """
    Load MNE objects for a given subject and output name, with an option to load only rescaled high gamma epochs.

    Parameters:
    - sub (str): Subject identifier.
    - output_name (str): Output name used in the file naming.
    - task (str): Task identifier.
    - just_HG_ev1_rescaled (bool): If True, only the rescaled high gamma epochs are loaded.
    - LAB_root (str, optional): Root directory for the lab. If None, it will be determined based on the OS.

    Returns:
    A dictionary containing loaded MNE objects.
    """

    # Determine LAB_root based on the operating system
    if LAB_root is None:
        HOME = os.path.expanduser("~")
        LAB_root = os.path.join(HOME, "Box", "CoganLab") if os.name == 'nt' else os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

    # Get data layout
    layout = get_data(task, root=LAB_root)
    save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs', sub)

    # Ensure save directory exists
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    # Initialize the return dictionary
    mne_objects = {}

    if just_HG_ev1_rescaled:
        # Define path and load only the rescaled high gamma epochs
        HG_ev1_rescaled_file = f'{save_dir}/{sub}_{output_name}_HG_ev1_rescaled-epo.fif'
        HG_ev1_rescaled = mne.read_epochs(HG_ev1_rescaled_file)
        mne_objects['HG_ev1_rescaled'] = HG_ev1_rescaled
    else:
        # Define file paths
        HG_ev1_file = f'{save_dir}/{sub}_{output_name}_HG_ev1-epo.fif'
        HG_base_file = f'{save_dir}/{sub}_{output_name}_HG_base-epo.fif'
        HG_ev1_rescaled_file = f'{save_dir}/{sub}_{output_name}_HG_ev1_rescaled-epo.fif'
        
        # Load the objects
        HG_ev1 = mne.read_epochs(HG_ev1_file)
        HG_base = mne.read_epochs(HG_base_file)
        HG_ev1_evoke = HG_ev1.average(method=lambda x: np.nanmean(x, axis=0))
        HG_ev1_rescaled = mne.read_epochs(HG_ev1_rescaled_file)
        HG_ev1_evoke_rescaled = HG_ev1_rescaled.average(method=lambda x: np.nanmean(x, axis=0))

        mne_objects['HG_ev1'] = HG_ev1
        mne_objects['HG_base'] = HG_base
        mne_objects['HG_ev1_evoke'] = HG_ev1_evoke
        mne_objects['HG_ev1_rescaled'] = HG_ev1_rescaled
        mne_objects['HG_ev1_evoke_rescaled'] = HG_ev1_evoke_rescaled

    return mne_objects


def create_subjects_mne_objects_dict(subjects, output_names_conditions, task, combined_data, acc_array, just_HG_ev1_rescaled=False, LAB_root=None):
    """
    Adjusted to handle multiple conditions per output name, with multiple condition columns.

    Parameters:
    - subjects: List of subject IDs.
    - output_names_conditions: Dictionary where keys are output names and values are dictionaries
        of condition column names and their required values.
    - task: Task identifier.
    - combined_data: DataFrame with combined behavioral and trial information.
    - acc_array: dict of numpy arrays of 0 for incorrect and 1 for correct trials for each subject
    - LAB_root: Root directory for data (optional).
    """
    subjects_mne_objects = {}

    for sub in subjects:
        print(f"Loading data for subject: {sub}")
        sub_mne_objects = {}
        for output_name, conditions in output_names_conditions.items():
            print(f"  Loading output: {output_name} with conditions: {conditions}")
            
            # Build the filtering condition
            sub_without_zeroes = "D" + sub[1:].lstrip('0') 
            condition_filter = (combined_data['subject_ID'] == sub) # this previously indexed using sub_without_zeroes, but now just uses sub. 3/17.
                    
            for condition_column, condition_value in conditions.items():
                if isinstance(condition_value, list):
                    # If the condition needs to match any value in a list
                    condition_filter &= combined_data[condition_column].isin(condition_value)
                else:
                    # If the condition is a single value
                    condition_filter &= (combined_data[condition_column] == condition_value)
            
            # Filter combinedData for the specific subject and conditions
            subject_condition_data = combined_data[condition_filter]
            
            # Load MNE objects and update with accuracy data
            mne_objects = load_mne_objects(sub, output_name, task, just_HG_ev1_rescaled=just_HG_ev1_rescaled, LAB_root=None)
            
            if sub in acc_array:
                trial_counts = subject_condition_data['trialCount'].values.astype(int)
                accuracy_data = [acc_array[sub][i-1] for i in trial_counts if i-1 < len(acc_array[sub])] # Subtract 1 here for zero-based indexing in acc array.
                # Now pass trial_counts along with accuracy_data
                mne_objects['HG_ev1_rescaled'] = utils.add_accuracy_to_epochs(mne_objects['HG_ev1_rescaled'], accuracy_data)

            sub_mne_objects[output_name] = mne_objects
        subjects_mne_objects[sub] = sub_mne_objects

    return subjects_mne_objects

In [ ]:
output_names = ["Stimulus_i25s25_fixationCrossBase_1sec_mirror_0to1Test", "Stimulus_i25s75_fixationCrossBase_1sec_mirror_0to1Test", 
                "Stimulus_i75s25_fixationCrossBase_1sec_mirror_0to1Test", "Stimulus_i75s75_fixationCrossBase_1sec_mirror_0to1Test", 
                "Stimulus_i25r25_fixationCrossBase_1sec_mirror_0to1Test", "Stimulus_i25r75_fixationCrossBase_1sec_mirror_0to1Test", 
                "Stimulus_i75r25_fixationCrossBase_1sec_mirror_0to1Test", "Stimulus_i75r75_fixationCrossBase_1sec_mirror_0to1Test", 
                "Stimulus_c25s25_fixationCrossBase_1sec_mirror_0to1Test", "Stimulus_c25s75_fixationCrossBase_1sec_mirror_0to1Test", 
                "Stimulus_c75s25_fixationCrossBase_1sec_mirror_0to1Test", "Stimulus_c75s75_fixationCrossBase_1sec_mirror_0to1Test",
                "Stimulus_c25r25_fixationCrossBase_1sec_mirror_0to1Test", "Stimulus_c25r75_fixationCrossBase_1sec_mirror_0to1Test",
                "Stimulus_c75r25_fixationCrossBase_1sec_mirror_0to1Test", "Stimulus_c75r75_fixationCrossBase_1sec_mirror_0to1Test",]
                
output_names_conditions = {
    "Stimulus_i25s25_fixationCrossBase_1sec_mirror_0to1Test": {
        "congruency": "i",
        "congruencyProportion": "75%",
        "switchType": "s",
        "switchProportion": "25%"
    },
    "Stimulus_i25s75_fixationCrossBase_1sec_mirror_0to1Test": {
        "congruency": "i",
        "congruencyProportion": "75%",
        "switchType": "s",
        "switchProportion": "75%"
    },
    "Stimulus_i75s25_fixationCrossBase_1sec_mirror_0to1Test": {
        "congruency": "i",
        "congruencyProportion": "25%",
        "switchType": "s",
        "switchProportion": "25%"
    },
    "Stimulus_i75s75_fixationCrossBase_1sec_mirror_0to1Test": {
        "congruency": "i",
        "congruencyProportion": "25%",
        "switchType": "s",
        "switchProportion": "75%"
    },
    "Stimulus_i25r25_fixationCrossBase_1sec_mirror_0to1Test": {
        "congruency": "i",
        "congruencyProportion": "75%",
        "switchType": "r",
        "switchProportion": "25%"
    },
    "Stimulus_i25r75_fixationCrossBase_1sec_mirror_0to1Test": {
        "congruency": "i",
        "congruencyProportion": "75%",
        "switchType": "r",
        "switchProportion": "75%"
    },
    "Stimulus_i75r25_fixationCrossBase_1sec_mirror_0to1Test": {
        "congruency": "i",
        "congruencyProportion": "25%",
        "switchType": "r",
        "switchProportion": "25%"
    },
    "Stimulus_i75r75_fixationCrossBase_1sec_mirror_0to1Test": {
        "congruency": "i",
        "congruencyProportion": "25%",
        "switchType": "r",
        "switchProportion": "75%"
    },
    "Stimulus_c25s25_fixationCrossBase_1sec_mirror_0to1Test": {
        "congruency": "c",
        "congruencyProportion": "75%",
        "switchType": "s",
        "switchProportion": "25%"
    },
    "Stimulus_c25s75_fixationCrossBase_1sec_mirror_0to1Test": {
        "congruency": "c",
        "congruencyProportion": "75%",
        "switchType": "s",
        "switchProportion": "75%"
    },
    "Stimulus_c75s25_fixationCrossBase_1sec_mirror_0to1Test": {
        "congruency": "c",
        "congruencyProportion": "25%",
        "switchType": "s",
        "switchProportion": "25%"
    },
    "Stimulus_c75s75_fixationCrossBase_1sec_mirror_0to1Test": {
        "congruency": "c",
        "congruencyProportion": "25%",
        "switchType": "s",
        "switchProportion": "75%"
    },
    "Stimulus_c25r25_fixationCrossBase_1sec_mirror_0to1Test": {
        "congruency": "c",
        "congruencyProportion": "75%",
        "switchType": "r",
        "switchProportion": "25%"
    },
    "Stimulus_c25r75_fixationCrossBase_1sec_mirror_0to1Test": {
        "congruency": "c",
        "congruencyProportion": "75%",
        "switchType": "r",
        "switchProportion": "75%"
    },
    "Stimulus_c75r25_fixationCrossBase_1sec_mirror_0to1Test": {
        "congruency": "c",
        "congruencyProportion": "25%",
        "switchType": "r",
        "switchProportion": "25%"
    },
    "Stimulus_c75r75_fixationCrossBase_1sec_mirror_0to1Test": {
        "congruency": "c",
        "congruencyProportion": "25%",
        "switchType": "r",
        "switchProportion": "75%"
    },                                                            
}

task='GlobalLocal'

# Assuming 'combined_data' is your DataFrame and 'subjects' is your list of subject IDs
subjects_mne_objects = create_subjects_mne_objects_dict(subjects, output_names_conditions, task="GlobalLocal", combined_data=combined_data, acc_array=acc_array, just_HG_ev1_rescaled=True)

# Save the subjects_mne_objects dictionary to a file
# joblib.dump(subjects_mne_objects, 'subjects_mne_objects.joblib')

In [ ]:
# Load the subjects_mne_objects from the file. This is not equal to when we create subjects_mne_objects directly...test this later. 5/6.
# subjects_mne_objects_test = joblib.load('subjects_mne_objects.joblib')

load stimulus significant channels

In [ ]:
sig_chans_per_subject = utils.get_sig_chans_per_subject(subjects, task='GlobalLocal', LAB_root=None)

get electrodes for each roi (definitions based on destrieux atlas)

In [ ]:
rois_dict = {
    # 'dlpfc': ["G_front_middle", "G_front_sup", "S_front_inf", "S_front_middle", "S_front_sup"],
    # 'acc': ["G_and_S_cingul-Ant", "G_and_S_cingul-Mid-Ant"],
    # 'parietal': ["G_parietal_sup", "S_intrapariet_and_P_trans", "G_pariet_inf-Angular", "G_pariet_inf-Supramar"],
    'lpfc': ["G_front_inf-Opercular", "G_front_inf-Orbital", "G_front_inf-Triangul", "G_front_middle", "G_front_sup", "Lat_Fis-ant-Horizont", "Lat_Fis-ant-Vertical", "S_circular_insula_ant", "S_circular_insula_sup", "S_front_inf", "S_front_middle", "S_front_sup"],
    'v1': ["G_oc-temp_med-Lingual", "S_calcarine", "G_cuneus"]
}

rois = list(rois_dict.keys())
electrodes_per_subject_roi, sig_electrodes_per_subject_roi = utils.make_sig_electrodes_per_subject_and_roi_dict(rois_dict, subjects_electrodestoROIs_dict, sig_chans_per_subject)

In [ ]:
sig_electrodes_per_subject_roi

get electrode counts for each roi (just for fun)

In [ ]:
total_electrodes_info = utils.calculate_total_electrodes(sig_electrodes_per_subject_roi, electrodes_per_subject_roi)
for roi, counts in total_electrodes_info.items():
    print(f"Total number of significant {roi} electrodes across all subjects:", counts['total_significant_electrodes'])
    print(f"Total number of {roi} electrodes across all subjects:", counts['total_electrodes'])

check if any subjects have a different sampling rate than 2048 Hz

In [ ]:
def check_sampling_rates(subjects_mne_objects):
    # This dictionary will store subjects with different sampling rates
    different_sampling_rates = {}
    
    # Iterate through each subject and their corresponding data
    for subject, data in subjects_mne_objects.items():
        # Access the specific Epochs object and check its sampling rate
        if 'Stimulus_i25s25_fixationCrossBase_1sec_mirror_0to1Test' in data:
            epochs = data['Stimulus_i25s25_fixationCrossBase_1sec_mirror_0to1Test']['HG_ev1_rescaled']
            sampling_rate = epochs.info['sfreq']
            
            # Check if the sampling rate is not 2048.0 Hz
            if sampling_rate != 2048.0:
                different_sampling_rates[subject] = sampling_rate
    
    return different_sampling_rates

# Assuming 'subjects_mne_objects' is your dictionary containing MNE objects for each subject
different_rates = check_sampling_rates(subjects_mne_objects)

# Print the results
if different_rates:
    print("Subjects with different sampling rates:")
    for subject, rate in different_rates.items():
        print(f"Subject {subject} has a sampling rate of {rate} Hz.")
else:
    print("All subjects have a sampling rate of 2048.0 Hz.")

get trial averaged data

In [ ]:
# Define time indices just for the function to work. Don't actually use time averaged data.
time_indices = {
    'firstHalfSecond': (2048, 3072),
    'secondHalfSecond': (3072, 4096),
    'fullSecond': (2048, 4096)
}
# Process the data
data_trialAvg_lists, data_trialStd_lists, _, overall_electrode_mapping, electrode_mapping_per_roi = utils.process_data_for_roi(
    subjects_mne_objects, output_names, rois, subjects, sig_electrodes_per_subject_roi, time_indices)

# need to figure out how to use the overall_electrode_mapping to get the electrode names back. Apparently I do this in the ANOVA in roi analysis. 5/5.

# Concatenate the data
concatenated_trialAvg_data = utils.concatenate_data(data_trialAvg_lists, rois, output_names)

In [ ]:
def compute_avg_std(data_dict):
    """
    Computes the mean and standard deviation across electrodes for each time point in each ROI and output name.

    Parameters:
    - data_dict (dict): A dictionary containing trial-averaged neural data (concatenated_trialAvg_data). It is expected to have a structure where each key is an output name,
      and its value is another dictionary. This nested dictionary should have ROIs as keys, and arrays of shape (n_electrodes, n_timepoints)
      as values.

    Returns:
    - tuple: A tuple containing two dictionaries:
        - mean_data: A dictionary with the same structure as input, but each array contains the mean across electrodes at each timepoint.
        - std_data: A dictionary with the same structure as input, but each array contains the standard deviation across electrodes at each timepoint.

    Example:
    mean_trialAvg_data, std_trialAvg_data = compute_avg_std(concatenated_trialAvg_data)
    print("Mean data for LPFC:", mean_trialAvg_data['Stimulus_i25s25_fixationCrossBase_1sec_mirror_0to1Test']['lpfc'])
    print("Standard deviation data for LPFC:", std_trialAvg_data['Stimulus_i25s25_fixationCrossBase_1sec_mirror_0to1Test']['lpfc'])
    """
    mean_data = {}
    std_data = {}
    for output_name, rois_data in data_dict.items():
        mean_data[output_name] = {}
        std_data[output_name] = {}
        for roi, data in rois_data.items():
            # Compute mean and std across the electrodes (axis=0)
            mean_data[output_name][roi] = np.mean(data, axis=0)
            std_data[output_name][roi] = np.std(data, axis=0)
    return mean_data, std_data


In [ ]:
# get mean and std across electrodes for each time point, in the already trial-averaged data.
mean_across_elecs_trialAvg_data, std_across_elecs_trialAvg_data = compute_avg_std(concatenated_trialAvg_data)

try kumar's suggestion of flattening the data to be more data driven 5/7  

In [ ]:
concatenated_trialAvg_data

#### Do RSA!  
1. Create time windows of 20 ms width, at 5 ms steps through the electrode and trial-averaged data.  
2. For each time window and ROI, calculate (1-Spearman correlation coefficient) between each condition. This will make a 16x16 neural RDM for each time window and ROI.  
3. For each neural RDM, find its correlation with each model RDM.
4. Do time perm cluster stats for significance

In [ ]:
import numpy as np
from scipy import stats
from scipy.ndimage import label

def create_time_windows(data, window_size_ms, step_size_ms, sampling_rate):
    window_size_samples = int(window_size_ms * sampling_rate / 1000)
    step_size_samples = int(step_size_ms * sampling_rate / 1000)
    total_samples = data.shape[0]
    windows = [(start, start + window_size_samples) for start in range(0, total_samples - window_size_samples + 1, step_size_samples)]
    return windows

def make_spearman_rdm(data_list):
    num_conditions = len(data_list)
    rdm = np.zeros((num_conditions, num_conditions))
    flattened_data = [data.flatten() if data.ndim > 1 else data for data in data_list]
    for i in range(num_conditions):
        for j in range(num_conditions):
            if i != j:
                correlation, _ = stats.spearmanr(flattened_data[i], flattened_data[j])
                rdm[i, j] = 1 - correlation
            else:
                rdm[i, j] = 0
    return rdm

def make_neural_rdms_whole_roi(data_dict, sampling_rate=2048.0, window_size_ms=100, step_size_ms=20):
    example_condition = next(iter(data_dict.values()))
    example_roi_array = next(iter(example_condition.values()))
    example_electrode_data = example_roi_array[0, :]
    sample_windows = create_time_windows(example_electrode_data, window_size_ms, step_size_ms, sampling_rate)

    all_rdms = {}
    roi_data = {}
    for output_name, rois in data_dict.items():
        for roi, electrode_time_data in rois.items():
            if roi not in roi_data:
                roi_data[roi] = []
            roi_data[roi].append(electrode_time_data)

    for roi, electrode_time_data_list in roi_data.items():
        all_rdms[roi] = []
        for window in sample_windows:
            windowed_data_list = [condition_array[:, window[0]:window[1]] for condition_array in electrode_time_data_list]
            rdm = make_spearman_rdm(windowed_data_list)
            all_rdms[roi].append((window, rdm))
    
    return all_rdms

def permutation_test_with_clustering_for_rsa(neural_rdms, model_rdm, num_permutations=10000, alpha=0.05):
    actual_correlations = []
    perm_correlations = np.zeros((num_permutations, len(neural_rdms)))

    for idx, (window, neural_rdm) in enumerate(neural_rdms):
        neural_rdm_tril = neural_rdm[np.tril_indices_from(neural_rdm, k=-1)]
        model_rdm_tril = model_rdm[np.tril_indices_from(model_rdm, k=-1)]
        
        # Compute correlations for the actual data
        correlation, _ = stats.spearmanr(neural_rdm_tril, model_rdm_tril)
        actual_correlations.append(correlation)
        
        for perm in range(num_permutations):
            shuffled_model_rdm_tril = np.random.permutation(model_rdm_tril)
            perm_correlation, _ = stats.spearmanr(neural_rdm_tril, shuffled_model_rdm_tril)
            perm_correlations[perm, idx] = perm_correlation

    threshold = np.percentile(perm_correlations, 100 *  (1-alpha))

    actual_correlations = np.array(actual_correlations)
    
    clusters, num_clusters = label(actual_correlations > threshold)
    cluster_sizes = np.array([np.sum(actual_correlations[clusters == cluster]) for cluster in range(1, num_clusters + 1)])
    
    max_permuted_cluster_sizes = np.zeros(num_permutations)
    for perm in range(num_permutations):
        perm_clusters, num_perm_clusters = label(perm_correlations[perm, :] > threshold)
        if num_perm_clusters > 0:
            max_permuted_cluster_sizes[perm] = max([np.sum(perm_correlations[perm, perm_clusters == cluster]) for cluster in range(1, num_perm_clusters + 1)])
    
    p_values = np.ones_like(actual_correlations)  # Default p-value to 1.0 (not significant)
    cluster_p_values = np.array([np.sum(max_permuted_cluster_sizes >= size) / num_permutations for size in cluster_sizes])
    
    for cluster_index in range(1, num_clusters + 1):
        p_values[clusters == cluster_index] = cluster_p_values[cluster_index - 1]
    
    return p_values

def calculate_rsa_with_permutation_testing(neural_rdms, model_rdms, rois, num_permutations=10000, alpha=0.05):
    rsa_results = {roi: {model_name: [] for model_name in model_rdms} for roi in rois}
    
    for roi in rois:
        for model_name, model_rdm in model_rdms.items():
            p_values = permutation_test_with_clustering_for_rsa(neural_rdms[roi], model_rdm, num_permutations, alpha)
            for idx, (window, neural_rdm) in enumerate(neural_rdms[roi]):
                neural_rdm_tril = neural_rdm[np.tril_indices_from(neural_rdm, k=-1)]
                model_rdm_tril = model_rdm[np.tril_indices_from(model_rdm, k=-1)]
                correlation, _ = stats.spearmanr(neural_rdm_tril, model_rdm_tril)
                rsa_results[roi][model_name].append((window, 1-correlation, p_values[idx] if idx < len(p_values) else np.nan))
    
    return rsa_results

def calculate_rsa_whole_roi(data_dict, model_rdms, rois, sampling_rate=2048.0, window_size_ms=100, step_size_ms=20, num_permutations=10000, alpha=0.05):
    neural_rdms = make_neural_rdms_whole_roi(data_dict, sampling_rate, window_size_ms, step_size_ms)
    rsa_results = calculate_rsa_with_permutation_testing(neural_rdms, model_rdms, rois, num_permutations, alpha)
    return rsa_results


In [ ]:
# congruency_model_rdm = {}
# congruency_model_rdm['congruency'] = model_rdms['congruency']

neural_rdms_by_samples_and_roi = make_neural_rdms_whole_roi(concatenated_trialAvg_data, sampling_rate=2048.0, window_size_ms=200, step_size_ms=20)
rois = list(neural_rdms_by_samples_and_roi.keys())
rsa_results = calculate_rsa_whole_roi(concatenated_trialAvg_data, model_rdms, rois, num_permutations=500, alpha=0.05)

# Save rsa results for each electrode to a file
rois_str = '_'.join(rois)
with open(f'rsa_results_whole_roi_{rois_str}.pkl', 'wb') as file:
    pickle.dump(rsa_results, file)

# Example: Print the RSA result for a specific ROI and model
print("RSA results for ROI 'lpfc' and model 'congruency':", rsa_results['lpfc']['congruency'])

In [ ]:
# Save rsa results for each electrode to a file
rois_str = '_'.join(rois)
with open(f'rsa_results_whole_roi_{rois_str}.pkl', 'wb') as file:
    pickle.dump(rsa_results, file)

In [ ]:
rsa_results['lpfc']['congruency']

try to do rsa on each electrode individually too 5/10

In [ ]:
def make_index_mapping(sig_electrodes_per_subject_roi):
    """
    Constructs a mapping from subject and electrode labels to indices for each ROI.
    """
    index_mapping = {}
    for roi, subjects in sig_electrodes_per_subject_roi.items():
        index_mapping[roi] = {}
        electrode_counter = 0  # Reset counter for each ROI
        for subject, electrodes in subjects.items():
            for electrode in electrodes:
                index_mapping[roi][electrode_counter] = (subject, electrode)
                electrode_counter += 1
    return index_mapping

def make_neural_rdms_per_electrode(data_dict, sig_electrodes_per_subject_roi, sampling_rate, window_size_ms, step_size_ms):
    """
    Makes a neural RDM across all conditions for each subject-specific electrode in each ROI and time window, using ROI-specific indexing.
    """

    example_condition = next(iter(data_dict.values()))  # Get the first condition's data
    example_roi = next(iter(example_condition.values()))  # Get the first ROI's data array from that condition
    example_electrode_data = example_roi[0, :]  # Get the first electrode (row) time series from the example roi array
    sample_windows = create_time_windows(example_electrode_data, window_size_ms, step_size_ms, sampling_rate)

    # Prepare the index mapping for accessing subject and electrode names
    index_mapping = make_index_mapping(sig_electrodes_per_subject_roi)

    # Initialize a dictionary to hold the RDMs
    all_rdms = {roi: {} for roi in sig_electrodes_per_subject_roi}
    # First organize the data by ROI, aggregating across all conditions
    roi_data = {}
    for output_name, rois in data_dict.items():
        for roi, electrode_time_data in rois.items():
            if roi not in roi_data:
                roi_data[roi] = []
            roi_data[roi].append(electrode_time_data)

    # Calculate RDMs for each electrode and time window
    for roi, electrode_time_data_list in roi_data.items():
        for window in sample_windows:
            for electrode_index in range(electrode_time_data_list[0].shape[0]):
                windowed_data_list = [condition_array[electrode_index, window[0]:window[1]] for condition_array in electrode_time_data_list]
                rdm = make_spearman_rdm(windowed_data_list)
                subject, electrode = index_mapping[roi][electrode_index]
                if subject not in all_rdms[roi]:
                    all_rdms[roi][subject] = {}
                if electrode not in all_rdms[roi][subject]:
                    all_rdms[roi][subject][electrode] = []
                all_rdms[roi][subject][electrode].append((window, rdm))

    return all_rdms

def calculate_rsa_per_electrode(neural_rdms, model_rdms, rois):
    """
    Calculates the RSA between neural RDMs and model RDMs for each electrode in each ROI, including subject-specific data.
    
    Parameters:
    - neural_rdms: dict
        A dictionary of neural RDMs structured by ROI, then subject, then electrode.
    - model_rdms: dict
        A dictionary of model RDMs structured by ROI.
    - rois: list
        List of ROIs to be analyzed.
    
    Returns:
    - dict
        A structured dictionary containing RSA results mapped by ROI, subject, electrode, and sample window.
    """
    rsa_results = {}

    for roi in rois:
        rsa_results[roi] = {}

        for model_name, model_rdm in model_rdms.items():
            rsa_results[roi][model_name] = {}

            for subject in neural_rdms[roi]:
                rsa_results[roi][model_name][subject] = {}

                for electrode in neural_rdms[roi][subject]:
                    rsa_results[roi][model_name][subject][electrode] = []

                    neural_rdm_windows = neural_rdms[roi][subject][electrode]
                    p_values = permutation_test_with_clustering_for_rsa(neural_rdm_windows, model_rdm, num_permutations=500, alpha=0.05)
                    
                    for idx, (window, neural_rdm) in enumerate(neural_rdm_windows):
                        # Extract the lower triangular part of the neural RDM, excluding the diagonal
                        tri_indices = np.tril_indices_from(neural_rdm, k=-1)
                        neural_rdm_tril = neural_rdm[tri_indices]
                        model_rdm_tril = model_rdm[tri_indices]
                        correlation, _ = stats.spearmanr(neural_rdm_tril, model_rdm_tril)
                        rsa_results[roi][model_name][subject][electrode].append((window, 1-correlation, p_values[idx] if idx < len(p_values) else np.nan))
               
    return rsa_results

In [ ]:
neural_rdms_by_samples_and_roi

In [ ]:
neural_rdms_by_sub_and_elec

In [ ]:
neural_rdms_by_sub_and_elec = make_neural_rdms_per_electrode(concatenated_trialAvg_data, sig_electrodes_per_subject_roi, sampling_rate=2048.0, window_size_ms=100, step_size_ms=20)

# Define ROIs based on keys in your neural_rdms_by_samples_and_roi (if they are consistent)
rois = list(neural_rdms_by_sub_and_elec.keys())

# Calculate RSA
rsa_results_by_sub_and_elec = calculate_rsa_per_electrode(neural_rdms_by_sub_and_elec, model_rdms, rois)


In [ ]:
rsa_results_by_sub_and_elec

In [ ]:
# Save rsa results for each electrode to a file
with open('rsa_results_by_sub_and_elec.pkl', 'wb') as file:
    pickle.dump(rsa_results_by_sub_and_elec, file)

In [ ]:
# # Load the rsa results for each electrode
# with open('rsa_results_by_sub_and_elec.pkl', 'rb') as file:
#     rsa_results_by_sub_and_elec = pickle.load(file)

In [ ]:
rsa_results_by_sub_and_elec

plot rsa results

In [ ]:
def plot_significance_rsa(ax, sig_windows, y_offset=0.1):
    """
    Plot significance bars for the effects on top of the existing axes, adjusted for time windows.

    Parameters:
    - ax: The matplotlib Axes object to plot on.
    - sig_windows: List of tuples where each tuple is (start_time, end_time, p_value).
    - y_offset: The vertical offset for placing significance bars.
    """
    y_pos_base = ax.get_ylim()[1]  # Get the top y-axis limit to place significance bars

    # Cluster contiguous significant windows
    clusters = []
    current_cluster = []

    for start_time, end_time, p_value in sig_windows:
        if not current_cluster:
            current_cluster = [start_time, end_time, p_value]
        else:
            # Check if the current window is contiguous with the last window in the cluster
            if start_time <= current_cluster[1]:
                current_cluster[1] = end_time
                current_cluster[2] = min(current_cluster[2], p_value)  # Use the most significant p_value
            else:
                clusters.append(tuple(current_cluster))
                current_cluster = [start_time, end_time, p_value]

    # Append the last cluster if it exists
    if current_cluster:
        clusters.append(tuple(current_cluster))

    for start_time, end_time, p_value in clusters:
        y_pos = y_pos_base + y_offset

        # Define the color and number of asterisks based on the p_value
        color = 'black'
        num_asterisks = '*' if p_value < 0.05 else '**' if p_value < 0.01 else '***' if p_value < 0.001 else ''
        
        if num_asterisks:  # Only plot bars for significant p-values
            # Draw the significance bar
            ax.plot([start_time, end_time], [y_pos, y_pos], color=color, lw=4)
            ax.text((start_time + end_time) / 2, y_pos, num_asterisks, ha='center', va='bottom', color=color)

def plot_rsa_correlation_by_time(rsa_results, model_name, roi, colors, sampling_rate=2048.0, time_shift=1000, significance_level=0.05, save_dir=None):
    """
    Plots RSA correlation over time for specified ROIs and model.

    Parameters:
    - rsa_results: dict
        A dictionary containing RSA results.
    - model_name: str
        The model name to plot RSA results for.
    - roi: string
        ROIs to be included in the plot.
    - colors: dict
        A dictionary mapping ROIs to plot colors.
    - sampling_rate: float, optional
        The sampling rate of the data in Hz. Default is 2048.0.
    - time_shift: int, optional
        Time in milliseconds to shift the x-axis. Default is 1000 ms.
    - significance_level: float, optional
        The significance level to determine significant windows. Default is 0.05.
    """
    # Create the figure and axis objects
    fig, ax = plt.subplots(figsize=(10, 6))

    if roi in rsa_results:
        times = []
        # correlations = []
        correlation_distances = []
        sig_windows = []
        for window, correlation_distance, p_value in rsa_results[roi][model_name]:
            # Calculate the middle of the window
            middle_sample = (window[0] + window[1]) / 2
            time_ms = (middle_sample * 1000 / sampling_rate) - time_shift  # Subtract the shift to align times
            times.append(time_ms)
            correlation_distances.append(correlation_distance)
            
            # Collect significant windows
            if p_value < significance_level:
                start_time = (window[0] * 1000 / sampling_rate) - time_shift
                end_time = (window[1] * 1000 / sampling_rate) - time_shift
                sig_windows.append((start_time, end_time, p_value))

        # Sort the data by time for plotting lines
        sorted_indices = np.argsort(times)
        times = np.array(times)[sorted_indices]
        correlation_distances = np.array(correlation_distances)[sorted_indices]

        # Plot each ROI's data with a specific color and connect points with lines
        ax.scatter(times, correlation_distances, color=colors[model_name], label=f'{roi}')
        ax.plot(times, correlation_distances, color=colors[model_name])  # This line connects the dots

        # Plot significance bars
        plot_significance_rsa(ax, sig_windows)

    # Adding labels and title
    ax.set_xlabel('Time from Stim Onset (ms)')
    ax.set_ylabel('1-ρ')
    ax.set_title(f'{model_name.capitalize()} RSA Correlation Distance in {roi}')
    # ax.legend()

    # Set x-axis limits to reflect the new time range
    ax.set_xlim(-1000, 1500)
    if save_dir:
        save_name = f"{roi}_{model_name}_RSA_whole_roi.png"
        plt.savefig(os.path.join(save_dir, save_name))
        plt.close(fig)
    # Show the plot
    else:
        plt.show()


def plot_rsa_correlation_by_electrode_grid(rsa_results, model_name, roi, colors, sampling_rate=2048.0, time_shift=1000, significance_level=0.05, grid_size=(4, 4), save_dir=None):
    """
    Plots RSA correlation over time for each electrode in a grid layout, for specified ROIs and model.

    Parameters:
    - rsa_results: dict
        A dictionary containing RSA results.
    - model_name: str
        The model name to plot RSA results for.
    - roi: string
        ROI to plot.
    - colors: dict
        A dictionary mapping ROIs to plot colors.
    - sampling_rate: float, optional
        The sampling rate of the data in Hz. Default is 2048.0.
    - time_shift: int, optional
        Time in milliseconds to shift the x-axis. Default is 1000 ms.
    - significance_level: float, optional
        The significance level to determine significant windows. Default is 0.05.
    - grid_size: tuple, optional
        The grid size (rows, columns) for the electrode plots. Default is (4, 4).
    - save_dir: str, optional
        Directory to save the plots. If None, plots are not saved. Default is None.

    """
    all_electrodes = []

    # Flatten the electrode data for this roi across all subjects
    for subject in rsa_results[roi][model_name]:
        electrodes = rsa_results[roi][model_name][subject]
        for electrode in electrodes:
            all_electrodes.append((roi, subject, electrode, electrodes[electrode]))

    num_electrodes = len(all_electrodes)
    num_plots = grid_size[0] * grid_size[1]
    num_grids = (num_electrodes + num_plots - 1) // num_plots

    for grid_idx in range(num_grids):
        fig, axes = plt.subplots(grid_size[0], grid_size[1], figsize=(20, 12))
        axes = axes.flatten()

        for plot_idx in range(num_plots):
            electrode_idx = grid_idx * num_plots + plot_idx
            if electrode_idx >= num_electrodes:
                axes[plot_idx].axis('off')
                continue

            roi, subject, electrode, electrode_data = all_electrodes[electrode_idx]
            times = []
            correlations = []
            sig_windows = []

            for window, correlation, p_value in electrode_data:
                middle_sample = (window[0] + window[1]) / 2
                time_ms = (middle_sample * 1000 / sampling_rate) - time_shift
                times.append(time_ms)
                correlations.append(correlation)

                if p_value < significance_level:
                    start_time = (window[0] * 1000 / sampling_rate) - time_shift
                    end_time = (window[1] * 1000 / sampling_rate) - time_shift
                    sig_windows.append((start_time, end_time, p_value))

            sorted_indices = np.argsort(times)
            times = np.array(times)[sorted_indices]
            correlations = np.array(correlations)[sorted_indices]

            ax = axes[plot_idx]
            ax.scatter(times, correlations, color=colors[model_name], label=f'{electrode}')
            ax.plot(times, correlations, color=colors[model_name])
            plot_significance_rsa(ax, sig_windows)

            ax.set_title(f'{roi} - Subject {subject}, Electrode {electrode}')
            ax.set_xlabel('Time from Stim Onset (ms)')
            ax.set_ylabel('1-ρ')
            # ax.axvline(x=0.5, color='k', linestyle='--', linewidth=1)
            ax.spines['top'].set_visible(False)
            ax.spines['right'].set_visible(False)

        plt.tight_layout()
        if save_dir:
            save_name = f"{roi}_{model_name}_single_electrodes_grid_{grid_idx + 1}.png"
            plt.savefig(os.path.join(save_dir, save_name))
            plt.close(fig)
        else:
            plt.show()


individual electrode grid plotting 5/19

In [ ]:
sig_electrodes_per_subject_roi

In [ ]:
model_rdms.keys()

In [ ]:
# Determine LAB_root based on the operating system
HOME = os.path.expanduser("~")
LAB_root = os.path.join(HOME, "Box", "CoganLab") if os.name == 'nt' else os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")
save_dir = os.path.join(LAB_root, 'BIDS-1.1_GlobalLocal', 'BIDS', 'derivatives', 'RSA', 'figs')

rois = ['lpfc', 'v1']

# Define colors for the model names
colors = {
    'congruency': 'red',
    'switchType': 'blue',
    'congruencyProportion': 'pink',
    'switchProportion': 'purple',
    'congruency:congruencyProportion': (0.5,0,0.5),
    'switchType:switchProportion': (0.6,1,0.6)
}

In [ ]:
# rsa_results_by_sub_and_elec is the result from calculate_rsa_per_electrode
for roi in rois:
    plot_rsa_correlation_by_electrode_grid(rsa_results_by_sub_and_elec, model_name='congruency', roi=roi, colors=colors, save_dir=save_dir)
    plot_rsa_correlation_by_electrode_grid(rsa_results_by_sub_and_elec, model_name='switchType', roi=roi, colors=colors, save_dir=save_dir)
    plot_rsa_correlation_by_electrode_grid(rsa_results_by_sub_and_elec, model_name='congruencyProportion', roi=roi, colors=colors, save_dir=save_dir)
    plot_rsa_correlation_by_electrode_grid(rsa_results_by_sub_and_elec, model_name='switchProportion', roi=roi, colors=colors, save_dir=save_dir)

In [ ]:
# Example: Plot the RSA result for the specified ROIs and model
for roi in rois:
    plot_rsa_correlation_by_time(rsa_results, 'congruency', roi, colors, save_dir=save_dir)
    plot_rsa_correlation_by_time(rsa_results, 'switchType', roi, colors, save_dir=save_dir)
    plot_rsa_correlation_by_time(rsa_results, 'congruencyProportion', roi, colors, save_dir=save_dir)
    plot_rsa_correlation_by_time(rsa_results, 'switchProportion', roi, colors, save_dir=save_dir)

### stats  
1. for each roi, for each neural rdm (time window), compute correlation between a randomly shuffled model rdm and neural rdm. Do this a lot of times (10,000 for now).
2. Find proportion of random correlations that are greater than the actual correlation between the model rdm and the neural rdm. That's the p-value.
3. Find clusters in time for each permutation where p<0.05. For each cluster get a cluster statistic (sum of correlation values within the cluster). Do this for the real data too.
4. Get the cluster statistic for the largest cluster in each of the fake data permutations. Make a distribution of this.
5. For each cluster in the real data, find its p-value as the proportion of fake data clusters that have a cluster statistic >= the real data cluster.


In [ ]:
# def permutation_test_rsa(neural_rdm, model_rdm, num_permutations=10000, threshold=0.05):
#     """
#     Performs a permutation test for RSA with cluster-based correction.
#     """
#     actual_correlation, _ = stats.spearmanr(neural_rdm.flatten(), model_rdm.flatten())
    
#     permuted_correlations = np.zeros(num_permutations)
#     for i in range(num_permutations):
#         shuffled_model_rdm = np.random.permutation(model_rdm.flatten())
#         permuted_corr, _ = stats.spearmanr(neural_rdm.flatten(), shuffled_model_rdm)
#         permuted_correlations[i] = permuted_corr

#     clusters, num_clusters = label(actual_correlation > threshold)
#     actual_cluster_size = np.sum(clusters > threshold)
    
#     permuted_cluster_sizes = np.zeros(num_permutations)
#     for i in range(num_permutations):
#         perm_clusters, num_perm_clusters = label(permuted_correlations[i] > threshold)
#         if num_perm_clusters > 0:
#             permuted_cluster_sizes[i] = max([np.sum(permuted_correlations[i][perm_clusters == cluster]) for cluster in range(1, num_perm_clusters + 1)])

#     p_value = np.sum(permuted_cluster_sizes >= actual_cluster_size) / num_permutations
    
#     return actual_correlation, p_value

now try to plot the average dissimilarity of condition pairs in the neural rdms over time
E.g., congruent-congruent, incongruent-congruent and incongruent-congruent    

this is untested and wrong. Need to index correctly. 5/18.

In [ ]:
# this is already defined for the model rdms but here it is again as a reminder. Delete once everything works. 5/18.
conditions = ["i25s25", "i25s75", "i75s25", "i75s75", "i25r25", "i25r75", "i75r25", "i75r75",
              "c25s25", "c25s75", "c75s25", "c75s75", "c25r25", "c25r75", "c75r25", "c75r75"]

# Extract specific features from each condition
congruency = [cond[0] for cond in conditions]
switch_types = [cond[-3] for cond in conditions]
congruency_proportion = [int(cond[1:3]) for cond in conditions]
switch_proportion = [int(cond[-2:]) for cond in conditions]


In [ ]:
# Function to compute the average correlation for condition comparisons
def compute_average_correlations(neural_rdm, condition_indices):
    n = len(condition_indices)
    avg_corr = np.zeros((n, n))
    for i in range(n):
        for j in range(i, n):
            avg_corr[i, j] = np.mean([neural_rdm[x, y] for x in condition_indices[i] for y in condition_indices[j]])
            avg_corr[j, i] = avg_corr[i, j]  # Since RDM is symmetric
    return avg_corr

# Function to plot the average correlations over time
def plot_neural_rdm_averages(rsa_results, model_name, rois, conditions, sampling_rate=2048.0, time_shift=1000):
    condition_indices = {
        "congruency": {
            (0, 0): [i for i, c in enumerate(congruency) if c == 'c'],
            (1, 1): [i for i, c in enumerate(congruency) if c == 'i'],
            (0, 1): [i for i, c in enumerate(congruency) if c == 'c'],
        },
        "switchType": {
            (0, 0): [i for i, s in enumerate(switch_types) if s == 'r'],
            (1, 1): [i for i, s in enumerate(switch_types) if s == 's'],
            (0, 1): [i for i, s in enumerate(switch_types) if s == 'r'],
        }
    }
    
    for roi in rois:
        for subject in rsa_results[roi][model_name]:
            fig, ax = plt.subplots(figsize=(10, 6))
            times = []
            avg_correlations = {cond: [] for cond in condition_indices[model_name].keys()}
            
            for electrode in rsa_results[roi][model_name][subject]:
                for window, neural_rdm, _ in rsa_results[roi][model_name][subject][electrode]:
                    middle_sample = (window[0] + window[1]) / 2
                    time_ms = (middle_sample * 1000 / sampling_rate) - time_shift
                    if time_ms not in times:
                        times.append(time_ms)
                    avg_corr = compute_average_correlations(neural_rdm, condition_indices[model_name])
                    for cond, indices in condition_indices[model_name].items():
                        avg_correlations[cond].append(avg_corr[indices[0], indices[1]])

            for cond, avg_corr in avg_correlations.items():
                avg_corr = np.mean(avg_corr, axis=0)
                ax.plot(times, avg_corr, label=f'{cond} Comparison')

            ax.set_xlabel('Time from Stim Onset (ms)')
            ax.set_ylabel('Average Spearman Corr. Coeff.')
            ax.set_title(f'{model_name.capitalize()} Average Correlation over Time by Condition for ROI {roi} and Subject {subject}')
            ax.legend()
            ax.set_xlim(-1000, 1500)
            plt.show()